In [1]:
import pandas as pd
import pyodbc

In [2]:
server_02_str = ("Driver={ODBC Driver 17 for SQL Server};"
            "Server=AACTASTPDDBVM02;"
            "Trusted_Connection=yes;")
cnxn = pyodbc.connect(server_02_str)

In [ ]:
#get data from server
sql23 = '''select *
         FROM [AccountabilityArchive].[Static].[PrelimV6StaticFile2023]
         where fiscalyear=2023'''
data23 = pd.read_sql(sql23 , cnxn)

In [ ]:
sql22 = '''select *
         FROM [AccountabilityArchive].[Static].[LiveStaticFile2022]
         where fiscalyear=2022'''
data22 = pd.read_sql(sql22 , cnxn)

In [ ]:
sql21 = '''select *
         FROM [REDATA].[dbo].[StaticFileData2021]
         where fiscalyear=2021'''
data21 = pd.read_sql(sql21 , cnxn)

In [ ]:
#close connection
cnxn.close()

# to do

* AssessmentSubtestTitle
* ethnicity







In [96]:
#rename cols to be consistent with DR.B preference
col_rename = {'SAISID':'ID'
               ,'FiscalYear': 'YEAR'
               ,'SchoolCode': 'SCHOOL_NUMBER'
               ,'SchoolName': 'SCHOOL_NAME'
               ,'DistrictCode': 'DISTRICT_NUMBER'
               ,'DistrictName': 'DISTRICT_NAME'
               ,'Subject': 'CONTENT_AREA'
               ,'Performance': 'ACHIEVEMENT_LEVEL'
               ,'StudentGrade': 'GRADE_ENROLLED'
               ,'ScaleScore': 'SCALE_SCORE'
               ,'FirstName': 'FIRST_NAME'
               ,'LastName': 'LAST_NAME'
               ,'FAY': 'SCHOOL_ENROLLMENT_STATUS'
               ,'DistrictFAY': 'DISTRICT_ENROLLMENT_STATUS'
               ,'Gender': 'GENDER'
               ,'Ethnicity': 'ETHNICITY'
               ,'IncomeEligibility1and2': 'FREE_REDUCED_LUNCH_STATUS'
               ,'SPED': 'SPED_STATUS'
               ,'EL': 'ELL_STATUS'}

ethinicity_dict = {'A':'Asian'
                ,'B':'Black/African American'
                ,'H':'Hispanic or Latino'
                ,'I':'American Indian or Alaska Native'
                ,'W':'White'
                ,'R':'Multiple Races'
                ,'P':'Native Hawaiian or Pacific Islander'
                ,'U':'Unknown'}

In [108]:

data = pd.DataFrame()
for df in [data21, data22, data23]:

    temp = df.copy()
    print(temp.FiscalYear.unique())
    
    #============== Subject formatting
    # change subject to str in 2021
    if 2021 in temp.FiscalYear.astype(int).unique():
        for SubN, SubStr in zip(['677','675'], ['MATH', 'ELA']):
            temp.Subject = temp.Subject.astype(str).str.replace(SubN, SubStr, regex=True)
    else:
        # change subject to str in 2022 and 2023
        for SubN, SubStr in zip(['Math','ELA'], ['MATH', 'ELA']):
            temp.Subject = temp.Subject.astype(str).str.replace(SubN, SubStr, regex=True)
    #only include math and ELA
    temp =temp[temp.Subject.isin(['MATH', 'ELA'])].copy()
    print(temp.shape, '---> ELA and Math records only')
    print(temp.Subject.unique())
    
    #============== Gender Formating
    # change gender to full words
    for GenderN, GenderStr in zip(['M','F', 'U'], ['Male', 'Female', 'Unknown']):
        temp['Gender'] = temp.Gender.astype(str).str.replace(GenderN, GenderStr, regex=True)
    print(temp.Gender.unique())
        
    #============== Exclude MSAA records
    #convert scale score to numeric to use to identify alty assessments since 2021 staticfile didn't have an alternative assessment identifier
    temp['ScaleScore'] = pd.to_numeric(temp['ScaleScore'], errors='coerce')
    if 2021 in temp.FiscalYear.astype(int).unique():
        temp = temp[~temp.ScaleScore.between(1200, 1290)].copy()
    else:
        temp = temp[temp.TestType==680].copy()
    print(temp.shape, '---> ALT assessment records removed')
    
    #============== Exclude non-tested students and grades other than 3-8
    temp['StudentGrade'] = pd.to_numeric(temp['StudentGrade'], errors='coerce')
    mask = (temp.ScaleScore.notnull()) & (temp.StudentGrade.between(3,8))
    temp = temp[mask].copy()
    print(temp.shape, '---> Tested in grades 3 to 8 only')
    print(temp.StudentGrade.unique())
    
    #=============== code FAY as binary
    for fay_col in ['FAY', 'DistrictFAY']:
        temp[fay_col] = pd.to_numeric(temp[fay_col], errors='coerce')
        temp[fay_col] = temp[fay_col].apply(lambda x: 1 if x>=1 else 0)
        
    #=============== format EThnicity
    temp.replace({'Ethnicity':ethinicity_dict}, inplace=True)
    
    #=============== keep cols in dict keys only
    temp = temp[list(col_rename.keys())]
    
    #=============== Make AssessmentSubtestTitle col
    temp['AssessmentSubtestTitle'] = temp.Subject + ' Grade '+ temp.StudentGrade.astype(int).astype(str)
        
    #=============== rename cols
    temp.rename(col_rename, axis=1, inplace=True)
    
    ##status check
    
    print(temp.shape, 'final shape \n') 
    
    # add data to main df
    data = pd.concat([data, temp], axis=0)


['2021']
(2191574, 60) ---> ELA and Math records only
['ELA' 'MATH']
['Female' 'Male' 'Unknown']
(2182233, 60) ---> ALT assessment records removed
(882169, 60) ---> Tested in grades 3 to 8 only
[3 5 7 4 6 8]
(882169, 20) final shape 

[2022.]
(1585130, 68) ---> ELA and Math records only
['ELA' 'MATH']
['Female' 'Male' 'Unknown']
(1412575, 68) ---> ALT assessment records removed
(1063480, 68) ---> Tested in grades 3 to 8 only
[5. 6. 4. 3. 8. 7.]
(1063480, 20) final shape 

[2023.]
(1555144, 62) ---> ELA and Math records only
['ELA' 'MATH']
['Male' 'Female' 'Unknown']
(1048878, 62) ---> ALT assessment records removed
(1048878, 62) ---> Tested in grades 3 to 8 only
[3. 7. 6. 5. 4. 8.]
(1048878, 20) final shape 



In [109]:
#make sure numeric cols are numeric
numeric_cols = ['ID', 'YEAR', 'SCHOOL_NUMBER','DISTRICT_NUMBER','ACHIEVEMENT_LEVEL', 'GRADE_ENROLLED', 'SCALE_SCORE', 'SCHOOL_ENROLLMENT_STATUS',
       'DISTRICT_ENROLLMENT_STATUS', 'FREE_REDUCED_LUNCH_STATUS', 'SPED_STATUS', 'ELL_STATUS']
for CN in numeric_cols:
    data[CN] = pd.to_numeric(data[CN], errors='coerce')
    
# deduplicate records to keep highest score per kid per year per subject (regardless of school)
data.sort_values(['YEAR', 'ID','CONTENT_AREA','GRADE_ENROLLED', 'SCALE_SCORE'], ascending=False, inplace=True)
data = data [~data.duplicated(['YEAR', 'ID','CONTENT_AREA', 'GRADE_ENROLLED'])].copy()

data.YEAR = data.YEAR.astype(int)
data['TEST_MODE'] = None

In [111]:
#export data
data.to_csv(r'C:\Users\yfahmy\OneDrive - Arizona Dept. of Education\Desktop\2023\Growth\9.19.23\Arizona_Data_LONG.csv', index=False, na_rep='')